In [12]:
import numpy as np 
import pandas as pd
from datetime import datetime

In [13]:
data_1min = pd.read_csv('AAPL_full_1min_adjsplit.csv')
data_1min.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1min['Date'] = pd.to_datetime(data_1min['Date'], format="%Y-%m-%d %H:%M:%S")
data_1hour = pd.read_csv('AAPL_full_1hour_adjsplit.csv')
data_1hour.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1min["Minute Log Return"] = np.log(data_1min['Close']).diff()

In [14]:
import pandas as pd

def calculate_hourly_volatility(dataframe):
   
    # Convert 'Date' column to datetime if not already
    if 'Date' in dataframe.columns and not pd.api.types.is_datetime64_any_dtype(dataframe['Date']):
        dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d %H:%M:%S")

    # Extract 'Date' and 'Minute_Log_Return' columns
    date_log_return_df = dataframe[['Date', 'Minute Log Return']]

    # Set 'Date' as the index
    date_log_return_df.set_index('Date', inplace=True)

    #Scaled volatility
    hourly_volatility = date_log_return_df.resample('H').apply(lambda x: (x ** 2).sum(axis=0) / x.notna().sum(axis=0)).apply(lambda x: x ** 0.5)

    hourly_volatility.reset_index(inplace=True)
    hourly_volatility.columns = ['Date', 'Hourly_Volatility']

    return hourly_volatility


result_df = calculate_hourly_volatility(data_1min)

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/802173827.py:16: RuntimeWarning: invalid value encountered in double_scalars
  hourly_volatility = date_log_return_df.resample('H').apply(lambda x: (x ** 2).sum(axis=0) / x.notna().sum(axis=0)).apply(lambda x: x ** 0.5)


In [15]:
def remove_zero_volatility(dataframe):

    # Filter rows where 'Intraday_Volatility' is not zero
    filtered_dataframe = dataframe[dataframe['Hourly_Volatility'] != 0]

    return filtered_dataframe

# Remove rows with zero intraday volatility
filtered_result_df = remove_zero_volatility(result_df)

# Display the resulting DataFrame
print(filtered_result_df)
newresults = filtered_result_df

                      Date  Hourly_Volatility
0      2005-01-03 08:00:00           0.001755
1      2005-01-03 09:00:00           0.002889
2      2005-01-03 10:00:00           0.001386
3      2005-01-03 11:00:00           0.000733
4      2005-01-03 12:00:00           0.000642
...                    ...                ...
167887 2024-02-28 15:00:00           0.000312
167888 2024-02-28 16:00:00           0.000297
167889 2024-02-28 17:00:00           0.000376
167890 2024-02-28 18:00:00           0.000249
167891 2024-02-28 19:00:00           0.000241

[167735 rows x 2 columns]


In [16]:
import pandas as pd

def synchronize_dataframes_by_date(df1, df2):

    df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')
    df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce')

    # Identify common dates by merging on the 'Date' column
    common_dates = pd.merge(df1[['Date']], df2[['Date']], on='Date', how='inner')
    
    # Filter both dataframes to only include the common dates
    df1_sync = df1[df1['Date'].isin(common_dates['Date'])]
    df2_sync = df2[df2['Date'].isin(common_dates['Date'])]

    return df1_sync, df2_sync

newresults_sync, data_1hour_sync = synchronize_dataframes_by_date(newresults, data_1hour)


/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/1952436155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')


In [17]:
data_1hour_sync['Date'] = pd.to_datetime(data_1hour_sync['Date'])
newresults_sync['Date'] = pd.to_datetime(newresults_sync['Date'])
data_1hour_sync.set_index('Date', inplace=True)
newresults_sync.set_index('Date', inplace=True)

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/2225684696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Date'] = pd.to_datetime(data_1hour_sync['Date'])
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/2225684696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newresults_sync['Date'] = pd.to_datetime(newresults_sync['Date'])


In [18]:
data_1hour_sync["Hourly Volatility"] = newresults_sync["Hourly_Volatility"]

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/2337339055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync["Hourly Volatility"] = newresults_sync["Hourly_Volatility"]


In [19]:
data_1hour_sync['Range'] = data_1hour_sync['High'] - data_1hour_sync['Low']
data_1hour_sync['Return'] = np.log(data_1hour_sync['Close']).diff()
data_1hour_sync['Return_Squared'] = np.square(data_1hour_sync['Return'])
data_1hour_sync = data_1hour_sync.dropna()

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/68887939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Range'] = data_1hour_sync['High'] - data_1hour_sync['Low']
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/68887939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1hour_sync['Return'] = np.log(data_1hour_sync['Close']).diff()
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_4555/68887939.py:3: SettingWithCopyWarning: 
A value is try

In [20]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [21]:
data = data_1hour_sync.dropna()
hourly_volatility = data["Hourly Volatility"]
df = pd.DataFrame({'hourly_volatility': hourly_volatility})

# Create a new column based on the conditions
df['target'] = np.where(df['hourly_volatility'] > df['hourly_volatility'].shift(1), 1, 0)

# The first day has no previous day to compare, so set the signal to 0
df['target'].fillna(0, inplace=True)
data['target'] = df['target']
data_full = data.dropna()

In [24]:
data_full = data_full[["Return_Squared", "Hourly Volatility", "target"]]

In [26]:
data_full.to_csv(r"Hourly Volatility Dataset.csv")